In [2]:
import datetime
import sys

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

import pandas as pd
from pandas_datareader import data
from itertools import islice

from scipy.signal import savgol_filter
import statsmodels
import statsmodels.api as sm
import scipy.stats as stats
import scipy.optimize as optimize
import numpy as np
#from mpl_finance import candlestick_ohlc
import matplotlib.pyplot as plt
#For the usual plots!
#matplotlib inline
#Makes graphs in jupyter interactive!
%matplotlib widget

#Removing warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
plt.close('all') 

In [14]:
#########                        Inputs
    #Tickers used
tickers = ['SPY', 'NVDA', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['AAPL', 'SPY', 'ISRG', 'NVDA', 'GM']
    #Data timeframe
startd = datetime.datetime(2016, 1, 1) 
endd_2 = datetime.datetime(2020, 1, 1)
    #Inputs BB
kc1input = 'Open'
kc1period = 15
kc1multiplier = 0.75 #1
kc1atr = 1
kc1atrbased = 'Open'
    #Inputs BB
bb2input = 'Open'
bb2period = 50
bb2multiplier = 2.2
bb4input = 'Open'
bb4period = 50
bb4multiplier = 1
    #Inputs ATR
atr1period = 25
atr1multiplier = 1
atr1plateau = 1
    #Inputs SMA
sma1input = 'Open'
sma1period = 2
sma2input = 'Open'
sma2period = 15 #15
sma3input = 'Open'
sma3period = 5
sma4input = 'Open'
sma4period = 10
######### 

######################           Functions
def stock_data(startd, endd, tickers=tickers):
    df = data.DataReader(tickers, 'yahoo', startd, endd).reset_index()
#df.set_index('Date', inplace=True)            <-- weggehaald zodat polyfit en backtest "i-1" werken
#uitrekenen van de daily return (cert)
    for ticker in tickers:
        df[('Return', ticker)] = df[('Close', ticker)].shift(1) - df[('Close', ticker)] 
    for ticker in tickers:
        df[('Percent_return', ticker)] = df[('Close', ticker)].pct_change()
    return df

def ehlers_simple_decycle(df, src, hpLength, t, name, upper, lower):
    #hp = highpass
    alphaArg = 2*np.pi / (hpLength*np.sqrt(2))
    alpha = 'Alpha_' + name
    hp = 'Hp_' + name
    decycler = 'Decycler_' + name
    decycleru = 'Decycleru_' + name
    decyclerl = 'Decyclerl_' + name
    df[(alpha, t)] = 0        
    df[(hp, t)] = 0   
    df[(decycler, t)] = 0 
    df[(decycleru, t)] = 0 
    df[(decyclerl, t)] = 0 
    for i in range(hpLength, len(df)):
        #Hp filter
        if np.cos(alphaArg) != 0:
            df.loc[i, (alpha, t)] = (np.cos(alphaArg)+np.sin(alphaArg)-1) / np.cos(alphaArg)
        else:
            df.loc[i, (alpha, t)] = df.loc[i-1, (alpha, t)]
        #Decycling::: src = the input!
        df.loc[i, (hp, t)] = np.square(1-(df.loc[i, (alpha, t)]/2)) * (src[i]-2 * src[i-1]+src[i-2]) + 2 * (1-df.loc[i, (alpha, t)]) * df.loc[i-1, (hp, t)] - np.square(1-df.loc[i, (alpha, t)]) * df.loc[i-2, (hp, t)]
        df.loc[i, (decycler, t)] = src[i] - df.loc[i, (hp, t)]
        df.loc[i, (decycleru, t)] = df.loc[i, (decycler, t)]*upper
        df.loc[i, (decyclerl, t)] = df.loc[i, (decycler, t)]*lower
    return df

def rvi(df, src, std_length, rvi_length):  #2, 6#
    std = src.rolling(std_length).std()
    std = [0 if pd.isna(x) else x for x in std]
    for i in range(1, len(df)):
        if df.loc[i, ('Close', t)] >= df.loc[i-1, ('Close', t)]:
            df.loc[i, ('Rvi_up', t)] = std[i]
        else:
            df.loc[i, ('Rvi_up', t)] = 0
        if df.loc[i, ('Close', t)] < df.loc[i-1, ('Close', t)]:
            df.loc[i, ('Rvi_down', t)] = std[i]
        else:
            df.loc[i, ('Rvi_down', t)] = 0
    for i in range(len(df)):
        up_ma = df[('Rvi_up', t)].ewm(rvi_length).mean()
        down_ma = df[('Rvi_down', t)].ewm(rvi_length).mean()
        df[('Rvi', t)] = (100*up_ma)/(up_ma + down_ma)
    return df[('Rvi', t)]
    
def bollinger_bands(df, src, length, multiple):
    mean_bb = df[(src, t)].rolling(length).mean()
    std  = df[(src, t)].rolling(length).std()
    upper_bb = mean_bb + (std*multiple)
    lower_bb = mean_bb - (std*multiple)
    return mean_bb, upper_bb, lower_bb   
    
def fibonacci_bollinger_bands(df, src, length, multiple):
    for i in range(1, len(df)):
        mean_fbb = src.rolling(length).mean()
        std = src.rolling(length).std()
        df.loc[i, ('Fbb_mid', t)] = mean_fbb[i]
        df.loc[i, ('Fbb_up1', t)] = mean_fbb[i] + (0.236 * std[i] * multiple)
        df.loc[i, ('Fbb_up2', t)] = mean_fbb[i] + (0.382 * std[i] * multiple)
        df.loc[i, ('Fbb_up3', t)] = mean_fbb[i] + (0.5 * std[i] * multiple)
        df.loc[i, ('Fbb_up4', t)] = mean_fbb[i] + (0.618 * std[i] * multiple)
        df.loc[i, ('Fbb_up5', t)] = mean_fbb[i] + (0.764 * std[i] * multiple)
        df.loc[i, ('Fbb_up6', t)] = mean_fbb[i] + (1 * std[i] * multiple)
        df.loc[i, ('Fbb_low1', t)] = mean_fbb[i] - (0.236 * std[i] * multiple)
        df.loc[i, ('Fbb_low2', t)] = mean_fbb[i] - (0.382 * std[i] * multiple)
        df.loc[i, ('Fbb_low3', t)] = mean_fbb[i] - (0.5 * std[i] * multiple)
        df.loc[i, ('Fbb_low4', t)] = mean_fbb[i] - (0.618 * std[i] * multiple)
        df.loc[i, ('Fbb_low5', t)] = mean_fbb[i] - (0.764 * std[i] * multiple)
        df.loc[i, ('Fbb_low6', t)] = mean_fbb[i] - (1 * std[i] * multiple)
    return df
    
#Functions containing Average True Range#
def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()
def atr(df, based, n):                                         #Open based;;; OG = Close based
    high = df[('High', t)]
    low = df[('Low', t)]
    based = df[(based, t)]
    df[('Tr0', t)] = abs(high - low)
    df[('Tr1', t)] = abs(high - based.shift())
    df[('Tr2', t)] = abs(low - based.shift())
    Tr = df[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    df[('Atr', t)] = Atr
    return Atr
#De Stoploss ATR implementatie:
def atr_stoploss(df, based, t , n, multiplier):       
    high = df[('High', t)]
    low = df[('Low', t)]
    based = df[(based, t)]
    df[('Tr0', t)] = abs(high - low)
    df[('Tr1', t)] = abs(high - based.shift())
    df[('Tr2', t)] = abs(low - based.shift())
    Tr = df[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    #Atr = wwma(Tr, n)
    #df[('Atr', t)] = Atr
    df[('Atr', t)] = wwma(Tr, n)
#De Stoploss 2xATR implementatie:
    #Stoploss = inp - (df[('Atr', t)] * multiplier)
    Stoploss = high - (df[('Atr', t)] * multiplier)
    return Stoploss

def keltner_channel(df, src, atr, length, multiple):  #using sma!    #USES ATR!!!!!
    mean_kc = df[(src, t)].rolling(length).mean()
    upper_kc = mean_kc + (df[(atr, t)]*multiple)
    lower_kc = mean_kc - (df[(atr, t)]*multiple)
    return mean_kc, upper_kc, lower_kc 
######################

#===~~~~====~~~~~~===~~~~====~~~~~~          Start             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time1 = datetime.datetime.now().time()
print(f'1. Script Started timestamp =     {time1}')

############                       DATA
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
############

print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!


###~~~~~~~~~~~~~~~~~~~~~~~~            HAVE TO CLEAN THESE UP FOR REDUDANCY  <<<<<<<<<<<<<<<<<<<<<<------------------

######################             SELL 1
###FBB, Open, 25, 2
for t in tickers:
    df = fibonacci_bollinger_bands(df, df[('Open', t)], 50, 1.5)
#FBB width
    df[('Fbb_width', t)] = (df[('Fbb_up6', t)] - df[('Fbb_low6', t)]) / df[('Fbb_mid', t)]
    df[('Fbb_width_sma', t)] = df[('Fbb_width', t)].rolling(50).mean()
###
#KC
    #df[('Atr_kc1', t)] = atr(df,kc1atrbased,kc1atr) #'Open', 1 
    #df[('Kc_mean', t)], df[('Kc_up', t)], df[('Kc_low', t)] = keltner_channel(df,kc1input,'Atr_kc1',kc1period,kc1multiplier) #df, df[('Open', t)], 'Atr_kc1',15, 1.5
    df[('Bb_mean', t)], df[('Bb_up', t)], df[('Bb_low', t)] = bollinger_bands(df, kc1input, kc1period, kc1multiplier)
#Stoploss
    df[('Stoploss_atr1', t)] = atr_stoploss(df, 'Low', t, atr1period, atr1multiplier)  
###Signals
    df[('Sell1_close_bb_up', t)] = 0
    df[('Sell1_stoploss_atr1_low', t)] = 0
    df[('Sell1_fbb_width_sma', t)] = 0
    df[('Sell1_close_kc_up', t)] = 0
    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss1', t)] = df.loc[x:i, ('Stoploss_atr1', t)].max()
        try:
            if df.loc[i, ('Stoploss1', t)] == df.loc[i-1, ('Stoploss1', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss1', t)] >= df.loc[i, ('Low', t)]:
            df.loc[i:i+atr1plateau, ('Sell1_stoploss_atr1_low', t)] = 1
            next(islice(loop, atr1plateau-1, atr1plateau-1), None)
            x=i
        if df.loc[i, ('Stoploss1', t)] < df.loc[i, ('Low', t)]:
            df.loc[i, ('Sell1_stoploss_atr1_low', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('Close', t)] >= df.loc[i, ('Fbb_up6', t)]:
            df.loc[i, ('Sell1_close_fbb_up6', t)] = 1
        if df.loc[i, ('Close', t)] >= df.loc[i, ('Bb_up', t)]:
            df.loc[i, ('Sell1_close_bb_up', t)] = 1
        if df.loc[i, ('Fbb_width', t)] > df.loc[i, ('Fbb_width_sma', t)]:
            df.loc[i, ('Sell1_fbb_width_sma', t)] = 1

###Bringing all together -- Sell 1
for t in tickers:
    df[('Sell1_signal', t)] = 0
    loop = iter(range(1, len(df)-1)) 
    for i in loop:
        if df.loc[i, ('Sell1_fbb_width_sma', t)] == 1:
            if df.loc[i, ('Sell1_close_bb_up', t)] == 1:
                if df.loc[i, ('Sell1_stoploss_atr1_low', t)] == 1:
                    x=i
                    y=0
                    while True:
                        if df.loc[x, ('Sell1_close_bb_up', t)] == 1:
                            df.loc[x, ('Sell1_signal', t)] = 0.0
                            x+=1
                            y+=1
                        if df.loc[x, ('Sell1_close_bb_up', t)] == 0:
                            df.loc[x, ('Sell1_signal', t)] = 1
                            next(islice(loop, y-1, y-1), None)
                            break 
                        
######################

############################################        SELL 2
for t in tickers:
#BB, Open 50, 2.25
    df[('Bb2_mean', t)], df[('Bb2_up', t)], df[('Bb2_low', t)] = bollinger_bands(df, bb2input, bb2period, bb2multiplier)
#SMA; Open, 5
    df[('SMA3', t)] = df[(sma3input, t)].rolling(sma3period).mean()
###Signals
    df[('Sell2_sma5_bb2_up', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i-1, ('Bb2_up', t)] <= df.loc[i-1, ('SMA3', t)]:
            if df.loc[i, ('Bb2_up', t)] > df.loc[i, ('SMA3', t)]:
                df.loc[i, ('Sell2_sma5_bb2_up', t)] = 1
        
###Bringing all together -- Sell 2
    df[('Sell2_signal', t)] = 0
    for i in range(1, len(df)-1):
        if df.loc[i, ('Sell2_sma5_bb2_up', t)] == 1:
            df.loc[i, ('Sell2_signal', t)] = 1
############################################

#############################################################      SELL 3
#
            
#############################################################    
#===~~~~====~~~~~~===~~~~====~~~~~~          Half            ===~~~~====~~~~~~===~~~~====~~~~~~ 
time2 = datetime.datetime.now().time()
print(f'2. Script Halfway timestamp =   {time2}')
print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!

#####################################################################################     BUY STRATS 4
####Buy
###Calculations
for t in tickers:
#SMA; Open, 2
    df[('SMA1', t)] = df[('Close', t)].rolling(2).mean()
#SMA; Open, 15
    df[('SMA2', t)] = df[(sma2input, t)].rolling(sma2period).mean()
#FBB width  ->  FBB, Open, 25, 2
    df = fibonacci_bollinger_bands(df, df[('Open', t)], 25, 2)
    df[('Fbb2_width', t)] = (df[('Fbb_up6', t)] - df[('Fbb_low6', t)]) / df[('Fbb_mid', t)]
###Signals
    df[('Buy1_signal', t)] = 0
    df[('Buy2_signal', t)] = 0
    for i in range(1, len(df)-1): 
        #if df.loc[i, ('Fbb2_width', t)] <= 0.15:
            if df.loc[i-1, ('Close', t)] < df.loc[i-1, ('SMA2', t)]:
                if df.loc[i, ('Close', t)] >= df.loc[i, ('SMA2', t)]:
                    df.loc[i, ('Buy1_signal', t)] = 1
                
##################################################################################### 

################################################        STRATEGIE
####Sell strategie
for t in tickers:
    df[('Strat_s', t)] = 0
    for i in range(len(df)-1):
        if df.loc[i, ('Sell1_signal', t)] == 1:
            df.loc[i, ('Strat_s', t)] = 1
       # if df.loc[i, ('Sell2_signal', t)] == 1:
        #    df.loc[i, ('Strat_s', t)] = 1

####Buy Strategie
for t in tickers:
    df[('Strat_b', t)] = 0
    for i in range(len(df)-1):
        if df.loc[i, ('Buy1_signal', t)] == 1:
            df.loc[i, ('Strat_b', t)] = 1
        if df.loc[i, ('Buy2_signal', t)] == 1:
            df.loc[i, ('Strat_b', t)] = 1

###Totale Strategie ------------------------------------------
for t in tickers:
    i = 0  #longest sma = 20!
    while i < len(df)-1:
        try:
            if df.loc[i, ('Strat_s', t)] == 1:
                df.loc[i, ('Strat', t)] = 0
                i+=1
                while True:
                    if df.loc[i, ('Strat_b', t)] == 0:
                        df.loc[i, ('Strat', t)] = 0
                        i+=1
                        if i == len(df)-1:
                            break
                    if df.loc[i, ('Strat_b', t)] == 1:
                        df.loc[i, ('Strat', t)] = 1
                        i+=1
                        break               
            if df.loc[i, ('Strat_s', t)] == 0:
                df.loc[i, ('Strat', t)] = 1
                i+=1
                if i == len(df)-1:
                    break
        except:
            pass
            
################################################

#===~~~~====~~~~~~===~~~~====~~~~~~          END             ===~~~~====~~~~~~===~~~~====~~~~~~ 
time2 = datetime.datetime.now().time()
print(f'3. Script Completed timestamp =   {time2}')
print(f'Data, amount of rows = {df.index.max()}')    #For checking if dataframe length stays consistent!

1. Script Started timestamp =     18:53:32.682252
Data, amount of rows = 1005
2. Script Halfway timestamp =   18:56:20.780752
Data, amount of rows = 1005
3. Script Completed timestamp =   18:58:23.030655
Data, amount of rows = 1005


In [15]:
df[('Strat_return', 'Total')] = 0
for t in tickers:
    df[('Percent_return', t)] = df[('Close', t)].pct_change()
    df[('Percent_return', 'Total')] = df[('Close', t)].pct_change()
    
    df[('Strat_return', t)] = df[('Strat', t)] * df[('Percent_return', t)]
    df[('Strat_return', 'Total')] += df[('Strat_return', t)].cumsum()
for t in tickers:
    fig, ax = plt.subplots(5, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 1.5, 0.5, 0.5], 'hspace':0.02})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)])
    ax[1].plot(df[('Strat', t)])
    ax[2].plot(df[('Percent_return', t)].cumsum(), linewidth=1)
    ax[2].plot(df[('Strat_return', t)].cumsum(), linewidth=1, color='r')
    ax[3].plot(df[('Strat_b', t)])
    ax[3].plot(df[('Buy1_signal', t)])
    ax[3].plot(df[('Buy2_signal', t)], color='black')
    ax[4].plot(df[('Strat_s', t)])
    ax[4].plot(df[('Sell1_signal', t)])
    ax[4].plot(df[('Sell2_signal', t)], color='g')
    r1=df[('Strat_return', t)].sum()
    r2=df[('Percent_return', t)].sum()
    print(f'Strat return {t} = {r1}')
    print(f'Hold return {t} = {r2}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return AAPL = 1.4417051670041885
Hold return AAPL = 1.1443231076046156


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return SPY = 0.6061492164769673
Hold return SPY = 0.5041826175961925


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return ISRG = 1.6484573931725612
Hold return ISRG = 1.3104874014466508


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return NVDA = 3.1701078292895883
Hold return NVDA = 2.376017962472737


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Strat return GM = 0.5738179788914587
Hold return GM = 0.2293339810891658


In [553]:
print(df.index.max())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

1005


Attributes       Date  Adj Close      Close       High        Low       Open  \
Symbols                     AAPL       AAPL       AAPL       AAPL       AAPL   
0          2016-01-04  24.323219  26.337500  26.342501  25.500000  25.652500   
1          2016-01-05  23.713696  25.677500  26.462500  25.602501  26.437500   
2          2016-01-06  23.249626  25.174999  25.592501  24.967501  25.139999   
3          2016-01-07  22.268383  24.112499  25.032499  24.107500  24.670000   
4          2016-01-08  22.386133  24.240000  24.777500  24.190001  24.637501   
5          2016-01-11  22.748621  24.632500  24.764999  24.334999  24.742500   
6          2016-01-12  23.078775  24.990000  25.172501  24.709999  25.137501   
7          2016-01-13  22.485413  24.347500  25.297501  24.325001  25.080000   
8          2016-01-14  22.977186  24.879999  25.120001  23.934999  24.490000   
9          2016-01-15  22.425385  24.282499  24.427500  23.840000  24.049999   
10         2016-01-19  22.316872  24.165001  24.662500  23.875000  24.602501   
11         2016-01-20  22.346884  24.197500  24.547501  23.355000  23.775000   
12         2016-01-21  22.233753  24.075001  24.469999  23.735001  24.264999   
13         2016-01-22  23.415861  25.355000  25.365000  24.592501  24.657499   
14         2016-01-25  22.958717  24.860001  25.382500  24.802500  25.379999   
15         2016-01-26  23.085699  24.997499  25.219999  24.517500  24.982500   
16         2016-01-27  21.568815  23.355000  24.157499  23.334999  24.010000   
17         2016-01-28  21.723509  23.522499  23.629999  23.097500  23.447500   
18         2016-01-29  22.473866  24.334999  24.334999  23.587500  23.697500   
19         2016-02-01  22.263767  24.107500  24.177500  23.850000  24.117500   
20         2016-02-02  21.813553  23.620001  24.010000  23.570000  23.855000   
21         2016-02-03  22.245296  24.087500  24.209999  23.520000  23.750000   
22         2016-02-04  22.424036  24.150000  24.332500  23.797501  23.965000   
23         2016-02-05  21.825132  23.504999  24.230000  23.422501  24.129999   
24         2016-02-08  22.054953  23.752501  23.924999  23.260000  23.282499   
25         2016-02-09  22.050303  23.747499  23.985001  23.482500  23.572500   
26         2016-02-10  21.883173  23.567499  24.087500  23.525000  23.980000   
27         2016-02-11  21.750851  23.424999  23.680000  23.147499  23.447500   
28         2016-02-12  21.818171  23.497499  23.625000  23.252501  23.547501   
29         2016-02-16  22.433323  24.160000  24.212500  23.652500  23.754999   
30         2016-02-17  22.776880  24.530001  24.552500  24.037500  24.167500   
31         2016-02-18  22.345116  24.065001  24.722500  24.022499  24.709999   
32         2016-02-19  22.294046  24.010000  24.190001  23.950001  24.000000   
33         2016-02-22  22.489033  24.219999  24.225000  23.980000  24.077499   
34         2016-02-23  21.980667  23.672501  24.125000  23.637501  24.100000   
35         2016-02-24  22.307974  24.025000  24.094999  23.330000  23.495001   
36         2016-02-25  22.461180  24.190001  24.190001  23.812500  24.012501   
37         2016-02-26  22.495996  24.227501  24.504999  24.145000  24.299999   
38         2016-02-29  22.444927  24.172501  24.557501  24.162500  24.215000   
39         2016-03-01  23.336325  25.132500  25.192499  24.355000  24.412500   
40         2016-03-02  23.387390  25.187500  25.222500  24.910000  25.127501   
41         2016-03-03  23.561485  25.375000  25.427500  25.112499  25.145000   
42         2016-03-04  23.912012  25.752501  25.937500  25.342501  25.592501   
43         2016-03-07  23.647383  25.467501  25.707500  25.240000  25.597500   
44         2016-03-08  23.452389  25.257500  25.440001  25.100000  25.195000   
45         2016-03-09  23.473282  25.280001  25.395000  25.067499  25.327499   
46         2016-03-10  23.484886  25.292500  25.559999  25.037500  25.352501   
47         2016-03-11  23.737915  25.565001  25.570000  25.375000  25.559999   


# Graphs of the signals

SELL 1, 2 en 3 graphs!

In [669]:
for t in tickers:
    fig, ax = plt.subplots(5, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5, 0.5, 0.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Close', t)], color='black')
    ax[0].plot(df[('Kc_up', t)], color='purple')
    ax[1].plot(df[('Sell1_fbb_width_sma', t)], color='red')
    ax[2].plot(df[('Sell1_stoploss_atr1_low', t)], color='red')
    ax[3].plot(df[('Sell1_close_kc_up', t)], color='red')
    ax[4].plot(df[('Sell1_signal', t)], color='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
####Buy 2; Long drop 
#MAYBE THIS ONE IS ENOUGH????
#Check for potential filters for the 2 buy moments, dont want it triggered at a sideways trend!
#BB4; == BB1; Open, 50, 1
for t in tickers:
    df[('Mean_bb4', t)], df[('Upper_bb4', t)], df[('Lower_bb4', t)] = bollinger_bands(df, df[(bb4input, t)], bb4period, bb4multiplier)
#SMA; Open, 5
    df[('SMA1', t)] = df[(sma1input, t)].rolling(sma1period).mean()
#SMA; Open, 10
    df[('SMA2', t)] = df[(sma2input, t)].rolling(sma2period).mean()
for t in tickers:
    df[('Buy2_signal', t)] = 0
    for i in range(1, len(df)-1):
        #if df.loc[i, ('Mean_bb4', t)] <= df.loc[i-1, ('Mean_bb4', t)]:
        if df.loc[i, ('SMA1', t)] > df.loc[i, ('Lower_bb4', t)] and df.loc[i-1, ('SMA1', t)] <= df.loc[i-1, ('Lower_bb4', t)]:
            df.loc[i, ('Buy2_signal', t)] = 1
        elif df.loc[i, ('SMA2', t)] >= df.loc[i, ('Mean_bb4', t)] and df.loc[i-1, ('SMA2', t)] < df.loc[i-1, ('Mean_bb4', t)]:
            df.loc[i, ('Buy2_signal', t)] = 1

In [496]:
tickers = ['SPY', 'NVDA', 'AAPL', 'MSFT', 'PYPL', 'ISRG', 'GM', 'INTC', 'PANW']
tickers = ['AAPL']
    #Data timeframe
startd = datetime.datetime(2016, 1, 1) 
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()
    #Inputs ATR
atr1period = 25
atr1multiplier = 1.25
atr1plateau = 1

def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()
    #return values.rolling(n).sum()/n
def atr(df, n):                                         #Open based;;; OG = Close based
    high = df[('High', t)]
    low = df[('Low', t)]
    close = df[('Open', t)]
    df[('Tr0', t)] = abs(high - low)
    df[('Tr1', t)] = abs(high - close.shift())
    df[('Tr2', t)] = abs(low - close.shift())
    Tr = df[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    df[('Atr', t)] = Atr
    return Atr
#De Stoploss ATR implementatie:
def atr_stoploss(df, based, t , n, multiplier):       
    high = df[('High', t)]
    low = df[('Low', t)]
    inp = df[(based, t)]
    df[('Tr0', t)] = abs(high - low)
    df[('Tr1', t)] = abs(high - inp.shift())
    df[('Tr2', t)] = abs(low - inp.shift())
    Tr = df[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    df[('Atr', t)] = wwma(Tr, n)
#De Stoploss 2xATR implementatie:
    #Stoploss = 
    Stoploss = high - (df[('Atr', t)] * multiplier)
    return Stoploss


#Stoploss
for t in tickers:
    df[('Stoploss_atr1', t)] = atr_stoploss(df, 'Low', t, atr1period, atr1multiplier)
    #df[('Stoploss_atr1_raw', t)] = atr_stoploss(df, 'Low', t, atr1period, atr1multiplier)
    #df[('Stoploss_atr1', t)] = df[('Stoploss_atr1_raw', t)].rolling(25).mean()
    df[('Sell1_stoploss_atr1_low', t)] = 0
    x = 0
    loop = iter(range(len(df)))
    for i in loop:
        df.loc[i, ('Stoploss1', t)] = df.loc[x:i, ('Stoploss_atr1', t)].max()
        try:
            if df.loc[i, ('Stoploss1', t)] == df.loc[i-1, ('Stoploss1', t)]:
                x=i
        except:
            pass
        if df.loc[i, ('Stoploss1', t)] >= df.loc[i, ('Low', t)]:
            df.loc[i:i+atr1plateau, ('Sell1_stoploss_atr1_low', t)] = 1
            next(islice(loop, atr1plateau-1, atr1plateau-1), None)
            x=i
        if df.loc[i, ('Stoploss1', t)] < df.loc[i, ('Low', t)]:
            df.loc[i, ('Sell1_stoploss_atr1_low', t)] = 0
            

In [498]:
for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Low', t)], color='black')
    ax[0].plot(df[('Stoploss1', t)], color='purple')
    #ax[0].plot(df[('Stoploss_atr1', t)], color='green')
    ax[1].plot(df[('Sell1_stoploss_atr1_low', t)], color='black')
    #ax[1].plot(df[('Sell1_signal', t)], color='red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [463]:
print(df)

Attributes       Date  Adj Close      Close       High        Low       Open  \
Symbols                     AAPL       AAPL       AAPL       AAPL       AAPL   
0          2016-01-04  24.323219  26.337500  26.342501  25.500000  25.652500   
1          2016-01-05  23.713696  25.677500  26.462500  25.602501  26.437500   
2          2016-01-06  23.249626  25.174999  25.592501  24.967501  25.139999   
3          2016-01-07  22.268383  24.112499  25.032499  24.107500  24.670000   
4          2016-01-08  22.386133  24.240000  24.777500  24.190001  24.637501   
...               ...        ...        ...        ...        ...        ...   
1001       2019-12-24  70.234581  71.067497  71.222504  70.730003  71.172501   
1002       2019-12-26  71.628067  72.477501  72.495003  71.175003  71.205002   
1003       2019-12-27  71.600876  72.449997  73.492500  72.029999  72.779999   
1004       2019-12-30  72.025833  72.879997  73.172501  71.305000  72.364998   
1005       2019-12-31  72.552094  73.412

In [662]:
tickers = ['AAPL']
    #Data timeframe
startd = datetime.datetime(2016, 1, 1) 
endd_2 = datetime.datetime(2020, 1, 1)
df = data.DataReader(tickers, 'yahoo', startd, endd_2).reset_index()

def wwma(values, n):
    return values.ewm(alpha=1/n, adjust=False).mean()
def atr(df, based, n):                                         #Open based;;; OG = Close based
    high = df[('High', t)]
    low = df[('Low', t)]
    based = df[(based, t)]
    df[('Tr0', t)] = abs(high - low)
    df[('Tr1', t)] = abs(high - based.shift())
    df[('Tr2', t)] = abs(low - based.shift())
    Tr = df[[('Tr0', t), ('Tr1', t), ('Tr2', t)]].max(axis=1)
    Atr = wwma(Tr, n)
    df[('Atr', t)] = Atr
    return Atr
def keltner_channel(df, src, atr, length, multiple):  #using sma!    #USES ATR!!!!!
    mean_kc = df[(src, t)].rolling(length).mean()
    upper_kc = mean_kc + (df[(atr, t)]*multiple)
    lower_kc = mean_kc - (df[(atr, t)]*multiple)
    return mean_kc, upper_kc, lower_kc

for t in tickers:
    df[('Atr', t)] = atr(df, 'Open', 2)
    df[('Kc_mean', t)], df[('Kc_up', t)], df[('Kc_low', t)] = keltner_channel(df,'Open','Atr',15,1) #df, df[('Open', t)], 'Atr_kc1',15, 1.5


for t in tickers:
    fig, ax = plt.subplots(2, sharex=True, figsize=(10,5), gridspec_kw={'height_ratios':[2.5, 0.5], 'hspace':0.05})
    ax[0].set_title(t)
    ax[0].plot(df[('Open', t)])
    ax[0].plot(df[('Kc_mean', t)])
    ax[0].plot(df[('Kc_up', t)])
    ax[0].plot(df[('Kc_low', t)])
    ax[1].plot(df[('Atr', t)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …